# Funções Globais

In [ ]:
import numpy as np
import sympy as sp
from tqdm import tqdm
import plotly.graph_objects as go

def gradiente_simbolico(xs_simbolicos, objetivo_simbolico):
    return sp.Array([objetivo_simbolico.diff(x) for x in xs_simbolicos])

def funcao_gradiente(variaveis_simbolicas, gradiente_simbolico):
    return sp.lambdify(variaveis_simbolicas, gradiente_simbolico, 'numpy')

def metodo_gradiente(funcao_gradiente_numerico, x, c, l=None, alpha=0.0001, eps=0.0001, max_iter=10000):
    x_copia = x.copy()
    for _ in range(max_iter):
        if l:
            gradiente = funcao_gradiente_numerico(*x_copia,  c, l)
        else: 
            gradiente = funcao_gradiente_numerico(*x_copia,  c) 

        gradiente = gradiente.flatten()

        if np.linalg.norm(gradiente) < eps:
            break

        x_copia -= alpha * gradiente

    return x_copia

def penalidade(tipo, objetivo_simbolico, xs_simbolicos, h, g, ponto_inicial, fator, eps=0.001, max_iter=10000):
    x = np.array(ponto_inicial).astype("float64")
    l_numerico = None
    c_numerico = None

    lista_x = [x]

    psi = None
    penalizacao_simbolico = None

    c = sp.symbols('c')
    l = sp.MatrixSymbol("l", len(h), 1)

    if tipo in ["externa", "quadratica", "lagrangiana"]:

        c_numerico = 1

        if tipo == "externa":

            psi = (h.dot(h) if len(h) else 0) + sum([sp.Max(0, restricao)**2 for restricao in g])
            penalizacao_simbolico = objetivo_simbolico + c * psi

        elif tipo == "quadratica":
            psi = h.dot(h)
            penalizacao_simbolico = objetivo_simbolico + c/2 * psi

        else:
            psi = h.dot(h)
            penalizacao_simbolico = objetivo_simbolico + l.dot(h) + c/2 * psi
            l_numerico = np.array([1]*len(h)).astype("float64")

    elif tipo == "interna":
        c_numerico = 512

        # Barreira logaritmica
        psi = -sum([sp.log(-gx) for gx in g])

        penalizacao_simbolico = objetivo_simbolico + c * psi

    gradiente_sym = gradiente_simbolico(xs_simbolicos, penalizacao_simbolico)

    funcao_grad_numerico = funcao_gradiente([*xs_simbolicos, c], gradiente_sym)

    if tipo == "lagrangiana":
        funcao_grad_numerico = funcao_gradiente([*xs_simbolicos, c, l], gradiente_sym)
    
    for _ in tqdm(range(max_iter)):
        print(c_numerico)
        print(x)
        novo_x = metodo_gradiente(funcao_grad_numerico, x, c_numerico, l_numerico)
        lista_x.append(novo_x)

        if np.linalg.norm(x - novo_x) < eps:
            break

        x = novo_x
        
        if tipo in ["externa", "quadratica", "lagrangiana"]:
            c_numerico *= fator
        elif tipo == "interna":
            c_numerico /= fator

        if tipo == "lagrangiana":
            psi_funcao = sp.lambdify(xs_simbolicos, h, "numpy")
            psi_valor = psi_funcao(*x)
            psi_dimensao_reduzida = psi_valor.flatten()

            l += c_numerico*psi_dimensao_reduzida

    return x, np.array(lista_x)

def plotar_funcao_objetivo(f, z_func, xs, nome):
    x_range = (-10, 10)
    y_range = (-10, 10)
    # Define the space for x1 and x2

    x1 = np.linspace(*x_range, 100)
    x2 = np.linspace(*y_range, 100)
    x1, x2 = np.meshgrid(x1, x2)
    x3 = z_func(*xs)  # apply the constraint

    # Calculate the values of the function for the grid of points
    Z = f(*xs)

    # Separate the points
    xs_first = xs[0]
    xs_last = xs[-1]
    xs_middle = xs[1:-1]

    Z_points_first = f(*xs_first)
    Z_points_last = f(*xs_last)
    Z_points_middle = np.array([f(*x) for x in xs_middle])

    # Create the plot
    fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                          go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                       mode='markers', marker=dict(size=6, color='green')),
                          go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                       mode='markers', marker=dict(size=6, color='red')),
                          go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                       mode='markers', marker=dict(size=10, color='blue'))])
    fig.update_layout(scene=dict(
                        xaxis_title='X1',
                        yaxis_title='X2',
                        zaxis_title='Z'),
                      title_text="Função Objetivo 3D Interativa")

    fig.write_html(f"{nome}.html")

    fig.show()


# 1A

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3 = sp.symbols('x1 x2 x3')
xs_simbolicos = sp.Array([x1, x2, x3])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[-2, 0, 0], [0, np.sqrt(5), 0], [1, 0, 0], [0, 1, 1], [-1, 0, -1]])
b_simbolico = sp.Matrix([7, 2, 7, -2, 5])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrições
h = sp.Matrix([x1 + x2 + x3 - 1])
g = sp.Matrix([])

# O ponto inicial deve violar as restrições na penalização externa.
x0 = [1, 1, 1]

x_ponto, x_lista = penalidade("externa", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
x_ponto

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2, x3):
    return objetivo(x1, x2, x3)

# Defina o espaço para x1, x2 e x3
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)
x3 = 1 - x1 - x2  # de acordo com a restrição

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2, x3)

xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("1a.html")

fig.show()


# 1B

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3 = sp.symbols('x1 x2 x3')
xs_simbolicos = sp.Array([x1, x2, x3])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[-2, 0, 0], [0, np.sqrt(5), 0], [1, 0, 0], [0, 1, 1], [-1, 0, -1]])
b_simbolico = sp.Matrix([7, 2, 7, -2, 5])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

h = sp.Matrix([])

g1x = x1 + x2 + x3 - 1
g2x = -g1x
g = sp.Matrix([g1x, g2x])

# O ponto deve estar dentro das restrições
x0 = [1, 1, 1]

x_ponto, x_lista = penalidade("interna", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2, x3):
    return objetivo(x1, x2, x3)

# Defina o espaço para x1, x2 e x3
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)
x3 = 1 - x1 - x2  # de acordo com a restrição

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2, x3)

xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("1b.html")

fig.show()

# 2B

In [ ]:
# Defina as variáveis simbólicas
x1, x2 = sp.symbols('x1 x2')
xs_simbolicos = sp.Array([x1, x2])

objetivo_simbolico = 2*x1 + 3*x2 + 4*x1**2 + 2*x1*x2 + x2**2
objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

h = sp.Matrix([])

g1x = -(x1 - x2)
g2x = x1 + x2 - 4
g3x = -(x1 - 3)
g = sp.Matrix([g1x, g2x, g3x])

x0 = [5, 5]

x_ponto, x_lista = penalidade("externa", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2):
    return objetivo(x1, x2)

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe o último ponto
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("2b.html")

fig.show()


# 2C

In [ ]:
# Defina as variáveis simbólicas
x1, x2 = sp.symbols('x1 x2')
xs_simbolicos = sp.Array([x1, x2])

objetivo_simbolico = 2*x1 + 3*x2 + 4*x1**2 + 2*x1*x2 + x2**2

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")


h = sp.Matrix([])

g1x = -(x1 - x2)
g2x = x1 + x2 - 4
g3x = -(x1 - 3)
g = sp.Matrix([g1x, g2x, g3x])

# Com outros pontos não funciona direito
x0 = [18, -24]

x_ponto, x_lista = penalidade("interna", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2):
    return objetivo(x1, x2)

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe o último ponto
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("2c.html")

fig.show()


# 3B

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3 = sp.symbols('x1 x2 x3')
xs_simbolicos = sp.Array([x1, x2, x3])

# Função objetivo
objetivo_simbolico = x1**3 - 6*x1**2 + 11*x1 + x3
objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

h = sp.Matrix([])

g1x = x1**2 + x2**2 - x3**2
g2x = -(x1**2 + x2**2 + x3**2 - 4)
g3x = x3 - 5
g4x = -x1
g5x = -(x2*x3)

g = sp.Matrix([g1x, g2x, g3x, g4x, g5x])

x0 = [-3, -3, -3]

x_ponto, x_lista = penalidade("externa", objetivo_simbolico, xs_simbolicos, h, g, x0, 1.2)

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Função objetivo
def f(x1, x2, x3):
    return objetivo(x1, x2, x3)

# Defina o espaço para x1, x2 e x3
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x3 = np.linspace(-10, 10, 100)
x1, x2, x3 = np.meshgrid(x1, x2, x3)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2, x3)

# Separe o último ponto
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("3b.html")

fig.show()


# 3C

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3, t = sp.symbols('x1 x2 x3 t')
variaveis_simbolicas = sp.Array([x1, x2, x3, t])
xs_simbolicos = variaveis_simbolicas[:-1]

# Restrições
g1x = x1**2 + x2**2 - x3**2
g2x = x1**2 + x2**2 + x3**2 - 4
g3x = 5 - x3
g4x = x1
g5x = x2*x3

g = sp.Matrix([g1x, g2x, g3x, g4x, g5x])

# Função de barreira logarítmica
psi = -sp.log(-g1x) - sp.log(-g2x) - sp.log(-g3x) - sp.log(-g4x) - sp.log(-g5x)

# Função de penalização
P = objetivo_simbolico - c * psi

x0 = [1, 1, 1]

x_ponto, x_lista = penalidade("interna", variaveis_simbolicas, x0, g, P, 2)

# 5A

In [ ]:
# Defina as variáveis simbólicas
x1, x2, c = sp.symbols('x1 x2 c')
variaveis_simbolicas = sp.Array([x1, x2, c])
xs_simbolicos = sp.Array([x1, x2])

objetivo_simbolico = x1 + x2

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrição de igualdade
h = sp.Matrix([x1**2 + x2**2 - 2])

# Psi minusculo
psi = h.dot(h)

# Função de penalização
P = objetivo_simbolico + c/2 * psi

x0 = [3, -2]

x_ponto, x_lista = penalidade("quadratica", variaveis_simbolicas, x0, h, P, 1.2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2):
    return objetivo(x1, x2)

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe o último ponto
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("5a.html")

fig.show()


In [ ]:
x_ponto

# 5B

In [ ]:
# Defina as variáveis simbólicas
from sympy import MatrixSymbol

x1, x2, c = sp.symbols('x1 x2 c')

# Restrição de igualdade
h = sp.Matrix([x1**2 + x2**2 - 2])

l_symbol = MatrixSymbol('l', len(h), 1)

l = sp.Matrix(l_symbol)

variaveis_simbolicas = sp.Array([x1, x2, l, c])
xs_simbolicos = sp.Array([x1, x2])
variaveis_do_gradiente = sp.Array([x1, x2, l])

objetivo_simbolico = x1 + x2

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Psi minusculo
psi = h.dot(h)

# Função de penalização
P = objetivo_simbolico + l.dot(h) + c/2 * psi

x0 = [3, -2]

x_ponto, x_lista = penalidade("lagrangiana", variaveis_simbolicas, x0, h, P, 2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2):
    return objetivo(x1, x2)

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe o último ponto
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("5b.html")

fig.show()


In [ ]:
import numpy as np
from scipy.optimize import minimize

# Define a função objetivo
def objective(x):
    x1, x2, x3 = x
    return x1**3 - 6*x1**2 + 11*x1 + x3

# Define as restrições
def constraint1(x):
    x1, x2, x3 = x
    return x1**2 + x2**2 - x3**2

def constraint2(x):
    x1, x2, x3 = x
    return -(x1**2 + x2**2 + x3**2 - 4)

def constraint3(x):
    _, _, x3 = x
    return x3 - 5

def constraint4(x):
    x1, _, _ = x
    return -x1

def constraint5(x):
    _, x2, x3 = x
    return -(x2*x3)

# Define as restrições no formato para o Scipy
constraints = [{'type': 'ineq', 'fun': constraint1},
               {'type': 'ineq', 'fun': constraint2},
               {'type': 'ineq', 'fun': constraint3},
               {'type': 'ineq', 'fun': constraint4},
               {'type': 'ineq', 'fun': constraint5}]

# Define o ponto inicial
x0 = np.array([1, 1, 1])

# Chama a função minimize
solution = minimize(objective, x0, method='SLSQP', constraints=constraints)

# Printa a solução
print(solution)
